In [1]:
%%capture
!pip install langchain openai

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Custom Prompt Templates

Custom prompt templates in LangChain allow you to dynamically generate prompts tailored to your specific needs.

You may need them if you want to customize instructions, examples, context or input for your model. To create one, specify input variables, implement a `format()` method, and expose the inputs through `input_variables`.

They give you flexibility to mold prompts however you need for your model.

You may need a custom prompt template if you want to:

 - Include unique instructions, examples or context for your model
 - Dynamically insert task-specific information into the prompt
 - Have full control over the prompt structure and formatting

To create a custom prompt template:

1. Specify input variables the template expects and expose them through the `input_variables` attribute

2. Implement a `format()` method that generates the prompt string


The key difference is that default prompt templates like `PromptTemplate` and `StringPromptTemplate` only allow inserting simple string substitutions into a template.

For example:

In [13]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

template = PromptTemplate.from_template("Hello {name}!")
prompt = template.format(name="Alice")

print(prompt)

Hello Alice!


This is useful for simple cases but doesn't allow you to dynamically customize the prompt structure based on your inputs.

The key difference is:

 - Default templates: Simple string substitution

 - Custom templates: Full programmatic control over prompt generation

Custom templates allow you to dynamically assemble prompts however you need for your particular use case.

# Let's see an example of this

The `FunctionExplainerPromptTemplate` class is designed to take a function's name as input, fetch its source code, and format it into a prompt.

This class is a custom template that takes a function (not just its name) as input and leverages the inspect module to fetch the function's source code.

It then formats this information into a prompt using the `PROMPT` template string.

This formatted prompt is expected to be used to solicit a natural language explanation of the function's behavior from a language model.

The class also validates that the only input variable provided is the actual function object.

In [4]:
import inspect
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)

# Template string for the prompt that will be sent to the language model
PROMPT = """Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}

Source Code:
{source_code}

Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        # Ensuring that the only input variable is 'function_name'
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the function's name and source code."""
        # Retrieve the source code of the provided function
        source_code = get_source_code(kwargs["function_name"])

        # Format the PROMPT string using the function name and its source code
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "function-explainer"

In [5]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)

Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code

Source Code:
def get_source_code(function_name):
    """Return the source code of the provided function."""
    # Using the inspect module to get the source code of the function
    return inspect.getsource(function_name)


Explanation:



In [9]:
from langchain.llms import OpenAI

llm = OpenAI()

llm(prompt)

'This function takes in the name of a function as an argument and returns the source code of that function using the inspect module.'

# Another example


In [23]:
# Template string for the algorithm optimization prompt
ALGO_PROMPT = """Given the algorithm below, suggest optimizations or potential \
improvements, and return the optimized code.

Algorithm Name: {algorithm_name}

Source Code:
{source_code}

Suggestions:
"""

class AlgorithmOptimizerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes an algorithm as input and formats the prompt template to request optimization suggestions."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "algorithm_function" not in v:
            raise ValueError("algorithm_function must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        """Format the prompt using the algorithm's name and source code."""

        # Retrieve the source code of the provided algorithm
        source_code = get_source_code(kwargs["algorithm_function"])

        # Format the ALGO_PROMPT string using the algorithm name and its source code
        prompt = ALGO_PROMPT.format(
            algorithm_name=kwargs["algorithm_function"].__name__,
            source_code=source_code
        )
        return prompt

    def _prompt_type(self) -> str:
        """Return the type of prompt."""
        return "algorithm-optimizer"


In [24]:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


In [25]:
# Instantiate the AlgorithmOptimizerPromptTemplate with the appropriate input variable
algo_optimizer = AlgorithmOptimizerPromptTemplate(input_variables=["algorithm_function"])

# Generate a prompt for the function "recursive_factorial"
prompt = algo_optimizer.format(algorithm_function=recursive_factorial)
print(prompt)


Given the algorithm below, suggest optimizations or potential improvements, and return the optimized code.

Algorithm Name: recursive_factorial

Source Code:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion."""
    if n == 0:
        return 1
    else:
        return n * recursive_factorial(n-1)


Suggestions:



In [26]:
print(llm(prompt))


1. Implement memoization to cache the results of already computed factorials. This will avoid unnecessary calculations and result in faster execution.

Optimized Code:
def recursive_factorial(n: int) -> int:
    """Calculate factorial of a number using recursion with memoization."""
    cache = {}
    if n in cache:
        return cache[n]
    elif n == 0:
        return 1
    else:
        result = n * recursive_factorial(n-1)
        cache[n] = result
        return result
